In [1]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def get_jsonparsed_data(url):
    """
    Sends a GET request to FMP's Earning Surprise API and returns the resulting data in a dictionary
    """
    # sending get request and saving the response as response object
    response = requests.get(url=url)
    data = json.loads(response.text)
    return data

In [3]:
# Read in nasdaq data and extract symbols column
nasdaq_list = pd.read_csv("nasdaq_list.csv")
# Pull API key from .env file
FMP_API_KEY = config("FMP_API_KEY")

nasdaq_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8202 entries, 0 to 8201
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      8202 non-null   object 
 1   Name        8202 non-null   object 
 2   Market Cap  7729 non-null   float64
 3   Country     7458 non-null   object 
 4   IPO Year    4820 non-null   float64
 5   Sector      6178 non-null   object 
 6   Industry    6195 non-null   object 
dtypes: float64(2), object(5)
memory usage: 448.7+ KB


In [4]:
# Filter missing out rows with missing data
# Filter reduces dataset from 8202 entries to 3156 entries
nasdaq_list_filtered = nasdaq_list.dropna()
nasdaq_list_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3156 entries, 0 to 8200
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      3156 non-null   object 
 1   Name        3156 non-null   object 
 2   Market Cap  3156 non-null   float64
 3   Country     3156 non-null   object 
 4   IPO Year    3156 non-null   float64
 5   Sector      3156 non-null   object 
 6   Industry    3156 non-null   object 
dtypes: float64(2), object(5)
memory usage: 197.2+ KB


In [5]:
symbols = nasdaq_list_filtered.Symbol

url_list = []
for idx, val in enumerate(symbols):
    url = "https://financialmodelingprep.com/api/v3/historical/earning_calendar/{}?limit=80&apikey={}".format(val, FMP_API_KEY)
    url_list.append(url)

In [6]:
# Call FMP API for each URL using Concurrent library
# Run takes 211 seconds ... be patient
with concurrent.futures.ThreadPoolExecutor() as executor:
    res = [executor.submit(get_jsonparsed_data, url) for url in url_list]
    concurrent.futures.wait(res)

In [7]:
res[3].result()

[{'date': '2021-10-28',
  'symbol': 'AAPL',
  'eps': 1.24,
  'epsEstimated': 1.24,
  'time': 'amc',
  'revenue': 83360000000.0,
  'revenueEstimated': 117986000000.0},
 {'date': '2021-07-27',
  'symbol': 'AAPL',
  'eps': 1.3,
  'epsEstimated': 1.0,
  'time': 'amc',
  'revenue': 81434000000.0,
  'revenueEstimated': 84581900000.0},
 {'date': '2021-04-28',
  'symbol': 'AAPL',
  'eps': 1.4,
  'epsEstimated': 0.99,
  'time': 'amc',
  'revenue': 89584000000.0,
  'revenueEstimated': 76920700000.0},
 {'date': '2021-01-27',
  'symbol': 'AAPL',
  'eps': 1.68,
  'epsEstimated': 1.41,
  'time': 'amc',
  'revenue': 111439000000.0,
  'revenueEstimated': 90099617021},
 {'date': '2020-10-29',
  'symbol': 'AAPL',
  'eps': 0.73,
  'epsEstimated': 0.7,
  'time': 'amc',
  'revenue': 64698000000.0,
  'revenueEstimated': 62039178082.19178},
 {'date': '2020-07-30',
  'symbol': 'AAPL',
  'eps': 0.65,
  'epsEstimated': 0.51,
  'time': 'amc',
  'revenue': 59685000000.0,
  'revenueEstimated': 46829769230.76923},


In [8]:
df = pd.DataFrame()
for x in range(len(symbols)):
    res_df = pd.DataFrame(res[x].result())
    df = pd.concat([df, res_df])

In [9]:
print(df)

          date symbol   eps epsEstimated time       revenue   revenueEstimated
0   2021-11-22      A  1.21         1.17  amc  0.000000e+00       1461300000.0
1   2021-08-17      A   1.1         0.99  amc  1.586000e+09       1659210000.0
2   2021-05-25      A  0.71         0.83  amc  1.525000e+09  1745481927.710843
3   2021-02-16      A  0.94         0.89  amc  1.548000e+09  1461033707.865169
4   2020-11-23      A  0.72         0.93  amc  1.483000e+09  1817870967.741935
..         ...    ...   ...          ...  ...           ...                ...
22  2016-05-12   ZYNE -0.49        -0.56  bmo  7.250000e+03        8285.714286
23  2016-03-14   ZYNE -0.62        -0.29  bmo  4.927500e+04       23047.983871
24  2015-11-11   ZYNE -0.66        -0.38  bmo  1.994070e+05         194815.125
25  2015-08-27   ZYNE   NaN          NaN  bmo  1.539000e+04                0.0
26  2015-08-05   ZYNE   NaN          NaN  bmo  1.539000e+04                0.0

[75540 rows x 7 columns]


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75540 entries, 0 to 26
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              75540 non-null  object 
 1   symbol            75540 non-null  object 
 2   eps               71820 non-null  object 
 3   epsEstimated      71419 non-null  object 
 4   time              75540 non-null  object 
 5   revenue           75540 non-null  float64
 6   revenueEstimated  74322 non-null  object 
dtypes: float64(1), object(6)
memory usage: 4.6+ MB


In [11]:
df.to_csv("historical_earnings_calendar.csv")